In [9]:
import cv2
import os
import numpy as np
from tqdm import tqdm

def read_coordinates(coordinate_file):
    """
    读取坐标文件，支持一个图片包含多个人脸的情况。
    """
    coordinates = {}
    with open(coordinate_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) >= 9:
                filename = parts[0]
                if filename not in coordinates:
                    coordinates[filename] = []
                coords = [round(float(coord)) for coord in parts[1:]]
                coordinates[filename].append(coords)
    return coordinates

def face_calculate(h0, w0, coordinates):
    """
    根据人脸关键点计算边界框。
    """
    leye_x, leye_y, reye_x, reye_y, nose_x, nose_y, mouth_x, mouth_y = coordinates
    center_x, center_y = nose_x, nose_y
    face_h = 2 * (mouth_y - (leye_y + reye_y) / 2)
    face_w = 2 * (reye_x - leye_x)
    upper = max(0, round(center_y - (face_h / 2) * 1.5))
    lower = min(h0, round(center_y + face_h / 2))
    left = max(0, round(center_x - face_w / 2))
    right = min(w0, round(center_x + face_w / 2))
    return upper, lower, left, right

def calculate_bounding_box(image, coordinates):
    """
    计算一个人脸的边界框。
    """
    y1, y2, x1, x2 = face_calculate(h0=image.shape[0], w0=image.shape[1], coordinates=coordinates)
    return [x1, y1, x2, y2]

def process_images(image_folder, coordinate_file, data_path, imagesave_path):
    """
    处理图像，保存每张图片的所有人脸边界框信息。
    """
    coordinates = read_coordinates(coordinate_file)
    images_faces_info = []
    count = 0
    for filename, faces in tqdm(coordinates.items()):
        image_path = os.path.join(image_folder, filename)
        if not os.path.exists(image_path):
            continue
        image = cv2.imread(image_path)
        cv2.imwrite(os.path.join(imagesave_path, f'{count}.jpg'),image)
        faces_info = [count]  
        for face_coords in faces:
            bounding_box = calculate_bounding_box(image, face_coords)
            faces_info.append(bounding_box)
        images_faces_info.append(faces_info)
        count+=1
    
    np.save(os.path.join(data_path, 'g_t_bounding_boxes.npy'), np.array(images_faces_info, dtype=object))

def main():
    image_folder = '../caltech/Caltech_WebFaces'
    coordinate_file = '../caltech/inferencing_gt.txt'
    data_path = '../data'
    is_path = '../data/infer_pics'
    if not os.path.exists(is_path):
        os.makedirs(is_path)
    process_images(image_folder, coordinate_file, data_path, is_path)

if __name__ == "__main__":
    main()


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:00<00:00, 117.17it/s]


In [10]:
bounding_box = np.load('../data/g_t_bounding_boxes.npy',allow_pickle=True)

In [11]:
bounding_box[15][1:]

[[113, 165, 141, 195], [176, 165, 200, 196], [228, 162, 246, 184]]